In [1]:
# Donchian Weekly Classic Trend Following System
import gta_prices
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'STAN'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p = [48, 24, 12, 6] # Look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [3]:
# Function definitions.
def charges(date, commission):
    '''Calculate trading charges.'''
    df = pd.DataFrame(index=date, columns=['date_shift', 'charges'])
    df.date_shift = df.index.values
    df.date_shift = df.date_shift.shift()
    df.charges = np.where(df.index == df.date_shift, 0, commission)
    return df.charges

def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame()
    df['upr'] = prices.high.rolling(period).max().shift(periods=1)
    df['lwr'] = prices.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df

def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

def state(entry_signal, exit_signal, period):
    '''Calculate trade state signals.'''
    df = pd.concat([entry_signal, exit_signal], axis=1)
    df.columns = ['entry', 'exit']
    df['state'] = 0
    for i in range(period, len(df)):
        if df.loc[df.index[i], 'entry'] == 1 \
        and df.loc[df.index[i - 1], 'state'] == 0:
            df.loc[df.index[i], 'state'] = 1
        elif df.loc[df.index[i], 'exit'] == 1:
            df.loc[df.index[i], 'state'] = 0
        else:
            df.loc[df.index[i], 'state'] = df.loc[df.index[i - 1], 'state']
    return df.state

def trade_summary(trade_list):
    '''Generate trade summary.'''
    frame = pd.DataFrame(columns=['trade', 'entry', 'volatility','cost', 'exit', 'days', 'profit', 'pct', 'annual'])
    if trade_list.entry.iloc[-1] == 1:
        trade_num = trade_list.index.max()
    else:
        trade_num = trade_list.index.max() + 1
        
    for i in list(range(1, trade_num)):
        df = trade_list.loc[i]
        trade = df.index[0]
        entry = df.date.iloc[0]
        volatility = df.volatility.iloc[0]
        cost = df.cost.iloc[0]
        exit = df.date.iloc[-1]
        days = df.days.iloc[-1]
        profit = df.profit.sum()
        pct = (profit / cost) * 100
        annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
        frame.loc[i] = [trade, entry, volatility, cost, exit, days, profit, pct, annual]
    frame = frame.set_index('trade')
    return frame

pd.set_option("display.max_columns", None)

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
for i in range(len(p)):
    globals()[f'p{i + 1}'] = p[i]
    globals()[f'dc{i + 1}'] = donchian(prices, p[i])

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = state(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = state(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = state(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = state(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# Trade list indexed by date.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = shares_div4(td.shares) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares), 0) # Adjust risk amount for revised share count.

In [12]:
# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [13]:
# Charges.
td['charges'] = charges(td.index, commission)

In [14]:
# Stamp duty.
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares), 0)

In [15]:
# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty), 0)

In [16]:
# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4)

In [17]:
# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges), 0)

In [18]:
# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

In [19]:
# Cumulative profit.
td['profit_cum'] = td.profit.cumsum()

In [20]:
# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100), 0)

In [21]:
# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

In [22]:
# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100)

In [23]:
# Trade list indexed by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td.round(2)

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,profit_cum,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2003-08-15,6.46,6.96,6.46,6.84,6.54,4.88,5.71,1,1,0,1,1,0,0.16,9108.88,1332,11.95,45.54,9166.38,0.00,0.00,0.00,0.00,0,0.00
1,2003-11-21,7.38,7.52,7.21,7.30,7.91,7.10,7.50,4,0,1,0,0,1,0.00,0.00,333,11.95,0.00,2291.59,2420.32,128.72,128.72,5.62,98,22.57
1,2004-01-02,7.35,7.52,7.35,7.47,7.91,7.10,7.50,3,0,1,0,0,1,0.00,0.00,333,11.95,0.00,2291.59,2476.43,184.83,313.55,8.07,140,22.41
1,2004-02-06,7.16,7.26,7.00,7.20,7.91,6.61,7.26,2,0,1,0,0,1,0.00,0.00,333,11.95,0.00,2291.59,2385.55,93.96,407.51,4.10,175,8.74
1,2004-07-16,6.99,7.06,6.93,6.98,7.91,6.46,7.18,1,0,1,0,0,1,0.00,0.00,333,11.95,0.00,2291.59,2310.72,19.13,426.64,0.83,336,0.91
2,2004-10-08,7.83,8.02,7.79,7.97,7.87,6.67,7.27,1,1,0,1,1,0,0.09,17175.13,2156,11.95,85.88,17272.95,0.00,0.00,426.64,0.00,0,0.00
2,2004-12-03,8.06,8.26,7.86,7.95,8.33,7.76,8.04,4,0,1,0,0,1,0.00,0.00,539,11.95,0.00,4318.24,4273.21,-45.03,381.61,-1.04,56,-6.60
2,2004-12-10,7.95,8.00,7.63,7.64,8.33,7.48,7.90,3,0,1,0,0,1,0.00,0.00,539,11.95,0.00,4318.24,4106.66,-211.58,170.03,-4.90,63,-25.25
2,2005-01-14,7.45,7.58,7.36,7.52,8.33,6.95,7.64,2,0,1,0,0,1,0.00,0.00,539,11.95,0.00,4318.24,4041.76,-276.48,-106.45,-6.40,98,-21.84


In [24]:
# Trade summary.
tds = trade_summary(td)
tds.round({'volatility': 3, 'pct': 1, 'annual': 1})

,entry,volatility,cost,exit,days,profit,pct,annual
trade,,,,,,,,
1,2003-08-15,0.165,9166.376410,2004-07-16,336,426.639790,4.7,5.1
2,2004-10-08,0.087,17272.952836,2005-03-18,161,-863.681436,-5.0,-11.0
3,2005-06-10,0.092,16460.810728,2006-06-09,364,2778.350072,16.9,16.9
4,2007-04-27,0.120,12595.555000,2007-08-24,119,-260.905000,-2.1,-6.2
5,2007-10-26,0.146,10378.983280,2008-01-18,84,-916.931280,-8.8,-33.1
6,2009-09-18,0.325,4647.283360,2011-01-28,497,32.604640,0.7,0.5
7,2013-01-11,0.180,8435.236900,2013-06-07,147,-358.387900,-4.2,-10.2
8,2016-10-21,0.251,6025.355440,2017-09-22,336,-325.969440,-5.4,-5.9
9,2019-07-05,0.160,9464.417200,2019-08-09,35,-1062.077200,-11.2,-71.1


In [25]:
cost_total = tds.cost.sum()
cost_total

112709.691554

In [26]:
profit_total = tds.profit.sum()
profit_total

-3314.908154000002

In [27]:
pct_total = (profit_total / cost_total) * 100
pct_total

-2.9411030305338084

In [28]:
days_total = (tds.exit.iloc[-1] - tds.entry.iloc[0])
days_total = days_total.days
days_total

6776

In [29]:
annual_total = ((1 + pct_total / 100) ** (365 / days_total) - 1) * 100
annual_total

-0.16067443089037292